<a href="https://colab.research.google.com/github/samuel-oluwemimo/ML_Exercise/blob/main/HOML_SKT_exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing data using svm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
impute = SimpleImputer(strategy='median')

In [8]:
df_train = train_set.copy()

In [9]:
df_1 = df_train.drop(['median_house_value','ocean_proximity'], axis=1)

In [10]:
impute.fit(df_1)

SimpleImputer(strategy='median')

In [11]:
X = impute.transform(df_1)

In [12]:
df_2 = pd.DataFrame(X, columns=df_1.columns)

In [13]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.0 MB


In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
oneHot = OneHotEncoder()

In [16]:
df_test = test_set.copy()
df_1hot = oneHot.fit_transform(df_test.copy()[['ocean_proximity']])

In [17]:
df_1hot.toarray()

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

# Support Vector Machine regressor

In [18]:
from sklearn.svm import SVR

In [19]:
svm_reg = SVR(kernel="linear")

In [20]:
train_label = train_set['median_house_value']
svm_reg.fit(df_2,train_label)

SVR(kernel='linear')

In [21]:
from sklearn.metrics import mean_squared_error

In [22]:
svm_predict = svm_reg.predict(df_2)
mse = mean_squared_error(train_label,svm_predict)
rmse = np.sqrt(mse)
rmse

93857.63712484317

In [23]:
from sklearn.model_selection import cross_val_score

In [24]:
scores = cross_val_score(svm_reg,df_2, train_label, cv=10, scoring='neg_mean_squared_error')
rmse  = np.sqrt(-scores)

In [25]:
def display_scores(scores):
  print(f'scores: {scores}')
  print(f'mean: {scores.mean()}')
  print(f'std: {scores.std()}')

display_scores(rmse)


scores: [94372.03592331 97254.50756046 94060.82169951 92387.42217054
 94614.53254577 94940.60822713 91976.89809449 93448.54116234
 98901.72000942 97678.71556393]
mean: 94963.58029568967
std: 2174.1369867065873


# Fine tuning the model

In [26]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

gridSearchSvm = GridSearchCV(svm_reg,param_grid, scoring='neg_mean_squared_error', return_train_score=True, cv=5)

gridSearchSvm.fit(df_2, train_label)

In [ ]:
negative_mse = gridSearchSvm.best_score_
rmse = np.sqrt(-negative_mse)
rmse

In [ ]:
gridSearchSvm.best_params_

# replacing grid search with randomized search